## TP3

Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en
Material_TPs/TP3/images a partir del template Material_TPs/TP3/template
1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos posit. os
2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo témplate del ítem 1
3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección

In [1]:
%matplotlib qt

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Vamos a probar con los 6 métodos de detección de patrones y observar en donde obtenemos falsos positivos para descartarlos. 
Comenzamos con la imagen logo_1.pngng"

In [39]:
# Cargamos la imagen a procesar
img = cv.imread('images/logo_1.png')

img_rgb= cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_gray= cv.cvtColor(img, cv.COLOR_BGR2GRAY)
template = cv.imread('template/pattern.png',0)

plt.figure()
plt.subplot(1,2,1)
plt.imshow(template, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_rgb)

In [41]:
w, h = template.shape[::-1]
# Los 6 métodos posibles para comparación:
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

for meth in methods:
    # Hago una copia de la imagen porque ciclo a ciclo le dibujo rectángulos
    img_salida = img_rgb.copy()
    
    method = eval(meth)
    
    # Aplicamos la coincidencia de patrones
    #--------------------------------------
    res = cv.matchTemplate(img_gray, template, method)
    
    # Encontramos los valores máximos y mínimos
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    
    # Si el método es TM_SQDIFF o TM_SQDIFF_NORMED, tomamos el mínimo
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    
    # Marcamos el lugar donde lo haya encontrado
    #----------------------------------------
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(img_salida,top_left, bottom_right, 255, 2)
    
    # Graficamos el procesamiento y la salida
    #----------------------------------------
    plt.figure()
    
    # Resultado de coincidencia
    plt.subplot(121),plt.imshow(res,cmap = 'gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    
    # Imagen original con recuadros
    plt.subplot(122),plt.imshow(img_salida)
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    
    plt.suptitle(meth)
    plt.show()

Podemos observar lo siguiente:
1. Con el metodo **Coeficiente de correlación cruzada** obtenemos un falso positivo parcial ya que al parecer detecta parte del  logo pero no llega a reconocerlo completamente.
2. Con el metodo **Coeficiente de correlación cruzada normalizado** observamos el mismo comportamiento que con TM_CCOEFF.
3. Con el metodo de **Correlación cruzada** observamos que tenemos una correcta deteccion salvo por una pequeña parte del logo que no llega a detectar.
4. Con el metodo de **Correlación cruzada normalizada** al igual que en TM_CCOEFF y TM_CCOEFF_NORMED observamos que solo detecta una parte del logo.
5. Con el metodo de **CDiferencia cuadrática media (SSD)** observamos que si hace una correcta detección del logo encerrando en el cuadro gran parte del logo pero dejando afuera una parte de la primera "C" de la palabra Coca Cola.
6. Con el metodo de **CDiferencia cuadrática media (SSD) normalizado** mismo resultado que con TM_SQDIFF

Hacemos lo mismo para "coca_logo_1".

In [47]:
# Cargamos la imagen a procesar
img = cv.imread('images/coca_logo_1.png')

img_rgb= cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_gray= cv.cvtColor(img, cv.COLOR_BGR2GRAY)
template = cv.imread('template/pattern.png',0)

plt.figure()
plt.subplot(1,2,1)
plt.imshow(template, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_rgb)

En el caso de esta imagen el codigo me da un error ya que la imagen es más pequeña que el template, lo que hago es primero hacer una detección de dicho problema para cada metodo e implemento un metodo para escalar la imagen del tamaño del template para poder implementar los metodos

In [49]:
def escalar_imagen_si_necesario(img_rgb, img_gray, template):
    h_img, w_img = img_gray.shape
    h_temp, w_temp = template.shape

    # Si la imagen es más pequeña que el template en cualquier dimensión
    if h_img < h_temp or w_img < w_temp:
        # Calculamos el factor necesario para igualar dimensiones
        scale_h = h_temp / h_img
        scale_w = w_temp / w_img
        scale_factor = max(scale_h, scale_w) + 0.05  # Un poco más para asegurar

        new_width = int(w_img * scale_factor)
        new_height = int(h_img * scale_factor)

        # Escalar ambas imágenes
        img_gray_scaled = cv.resize(img_gray, (new_width, new_height), interpolation=cv.INTER_LINEAR)
        img_rgb_scaled = cv.resize(img_rgb, (new_width, new_height), interpolation=cv.INTER_LINEAR)

        print(f"🔁 Imagen escalada automáticamente a: {new_width}x{new_height}")
        return img_rgb_scaled, img_gray_scaled
    else:
        return img_rgb, img_gray

In [51]:
w, h = template.shape[::-1]

# Los 6 métodos posibles para comparación:
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
           'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

img_rgb, img_gray = escalar_imagen_si_necesario(img_rgb, img_gray, template)

for meth in methods:
    img_salida = img_rgb.copy()
    method = eval(meth)

    # Verificación de tamaño
    if img_gray.shape[0] < template.shape[0] or img_gray.shape[1] < template.shape[1]:
        print(f"⚠️ El template es más grande que la imagen. Saltando método: {meth}")
        continue

    # Aplicamos la coincidencia de patrones
    res = cv.matchTemplate(img_gray, template, method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

    # Si el método es SQDIFF, buscamos el mínimo; en otros, el máximo
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc

    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(img_salida, top_left, bottom_right, 255, 2)

    # Mostrar resultados
    plt.figure(figsize=(10, 5))
    plt.subplot(121)
    plt.imshow(res, cmap='gray')
    plt.title('Matching Result')
    plt.xticks([]), plt.yticks([])

    plt.subplot(122)
    plt.imshow(img_salida)
    plt.title('Detected Point')
    plt.xticks([]), plt.yticks([])

    plt.suptitle(meth)
    plt.show()

🔁 Imagen escalada automáticamente a: 410x991
